**INIT PACKAGES**

In [8]:
from ultralytics import YOLO
import cv2
from paddleocr import PaddleOCR
import re
from collections import deque, Counter

**TRAIN MODEL**

In [2]:
model = YOLO("yolo12n.pt")

In [ ]:
model.train(data='C:/Jobs/ANPR/TrainingPt3/dataset/data.yaml', epochs=100)

**INITIALIZE INFERENCE**

In [2]:
model = YOLO("C:/Binus/Semester8/Periode1/DeepLearning/Project/ANPRProjectYoloOCR/runs/detect/train3/weights/best.pt")
ocr = PaddleOCR(use_angle_cls=True, lang='en')

PLATE_REGEX = r"^[A-Z]{1,2} ?\d{1,4} ?[A-Z]{2,3}$"

[2025/04/23 10:05:35] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\toras/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\toras/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320',

In [8]:
metrics = model.val()

Ultralytics 8.3.90  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv12n summary (fused): 159 layers, 2,557,118 parameters, 0 gradients, 6.3 GFLOPs


FileNotFoundError: 'C:/Jobs/ANPR/TrainingPt3/dataset/data.yaml' does not exist

**INFERENCE**

In [3]:
def normalize_plate(text):
    text = text.replace(" ", "").upper()
    match = re.match(r"^([A-Z]{1,2})(\d{1,4})([A-Z]{2,3})$", text)
    if match:
        return f"{match.group(1)} {match.group(2)} {match.group(3)}"
    return None

In [7]:
plate_history = deque(maxlen=15)
final_plate = ""

In [9]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # ==== DETEKSI PLAT ====
    results = model(frame)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            plate_img = frame[y1:y2, x1:x2]

            # OCR dengan PaddleOCR
            ocr_result = ocr.ocr(plate_img, cls=True)
            raw_text = ""

            if isinstance(ocr_result, list) and len(ocr_result) > 0:
                first_result = ocr_result[0]
                if isinstance(first_result, list) and len(first_result) > 0:
                    raw_text = first_result[0][1][0].strip()

            # Gambar bounding box hijau untuk plat
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Tampilkan raw OCR (kalau ada)
            if raw_text:
                cv2.putText(frame, raw_text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

                # Normalisasi & masukkan ke history
                normalized = normalize_plate(raw_text)
                if normalized:
                    plate_history.append(normalized)

    # ==== VOTING HASIL TERBAIK ====
    if plate_history:
        most_common_raw, count = Counter(plate_history).most_common(1)[0]
        if count >= 5:  # misal: tampilkan hanya jika muncul >=5x
            final_plate = most_common_raw

    # Tampilkan hasil akhir
    if final_plate:
        cv2.putText(frame, f"Detected: {final_plate}",
                    (10, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Tampilkan frame
    cv2.imshow("YOLOv8 + PaddleOCR + Voting", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 license-plat, 35.9ms
Speed: 4.3ms preprocess, 35.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
[2025/04/23 10:07:00] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.4093492031097412
[2025/04/23 10:07:00] ppocr DEBUG: cls num  : 0, elapsed : 0
[2025/04/23 10:07:00] ppocr DEBUG: rec_res num  : 0, elapsed : 0.0

0: 480x640 (no detections), 39.1ms
Speed: 3.8ms preprocess, 39.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 37.6ms
Speed: 4.1ms preprocess, 37.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 45.0ms
Speed: 3.5ms preprocess, 45.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 35.1ms
Speed: 1.6ms preprocess, 35.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license-plat, 38.1ms
Speed: 1.4ms preprocess, 38.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480,